#### 1- Import all required modules including the following:

In [2]:
from scikeras.wrappers import KerasClassifier 
from sklearn.model_selection import GridSearchCV

#### 2- Import the minst dataset, and split it into testing and training as we saw in the tutorial. Then, reshape each sample into a row vector, and scale it by dividing by 255.

In [3]:
# Keras will download MNIST digit dataset for us:
from keras.datasets import mnist

# By default, the first 60k of MNIST has been defined as training and the rest as testing set: 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [4]:
print(y_train)

[5 0 4 ... 5 6 8]


In [3]:
# Reshape and scale the training data
X_train = X_train.reshape(X_train.shape[0], 784)

# Reshape and scale the testing data
X_test = X_test.reshape(X_test.shape[0], 784)

print(X_train.shape)
print(X_test.shape)

(60000, 784)
(10000, 784)


In [4]:
# simply scale the features to the range of [0,1]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [5]:
# output label:
print (y_train.shape)
print (y_train[:10])

(60000,)
[5 0 4 1 9 2 1 3 1 4]


#### 3- Perform OneHotEncoding for the label y. So, your label will be a vector of 10 elements for each data sample.

In [6]:
from keras import utils as np_utils

# OneHotEncoding for the output label:

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [7]:
# Label after OneHotEncoding:
print (y_train.shape)
print (y_train[:10,:])

(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


#### 4- Now, define a function called model_creator. This function will define, create, and compile your neural network model according to your structure, and then return the built model as the output. For the ANN neurons/layers, use the same structure as we had in the tutorial:

In [8]:
# "Sequential" models let us define a stack of neural network layers
from keras import Sequential
# import the core layers:
from keras.layers import Dense, Dropout, Activation, Flatten

def model_creator():
    input_size = 784
    hidden_neurons = 100
    out_size = 10
    
    curr_model = Sequential()

    # first layer: input layer
    # Input layer does not do any processing, so no need to define the input layer in this problem.
    
    # -----------------------------------------
    # second layer: hidden layer:
    curr_model.add(Dense(hidden_neurons, input_dim = input_size))  # Neurons
    curr_model.add(Activation('sigmoid')) # Activation
    
    # -----------------------------------------
    # third layer: output layer:
    curr_model.add(Dense(out_size, input_dim = hidden_neurons))  # Neurons
    curr_model.add(Activation('softmax')) # Activation
    
    # compile:
    curr_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # return: 
    return curr_model 

#### 5- Fix the random state for reproducibility by adding the following in your code:    


In [9]:
import numpy as np

seed = 2
np.random.seed(seed)

#### 6- Use KerasClassifier class to wrap your model as an object:

In [146]:
model = KerasClassifier(model=model_creator, verbose=1)

#### 7- Now, run sklearn GridSearch to find the best batch_size and epochs. Search in this range: batch_size = [30 , 50 , 100 ] , epochs = [10 , 15 , 20].  In your GridSearch, the estimator is the above model,  the scoring should be 'neg_log_loss', and you have to use 10-fold CV.

In [147]:
param_grid = dict(batch_size=[30, 50, 100], epochs=[10, 15, 20])
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_log_loss', cv=10)

grid_result = grid.fit(X_train, y_train)

# view the best results corresponding to the best structure of ANN:
print(grid.best_score_)
print(grid.best_params_)

Epoch 1/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.4390 - accuracy: 0.8878
Epoch 2/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.2108 - accuracy: 0.9399
Epoch 3/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.1573 - accuracy: 0.9549
Epoch 4/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.1243 - accuracy: 0.9649
Epoch 5/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.1020 - accuracy: 0.9709
Epoch 6/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.0851 - accuracy: 0.9761
Epoch 7/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.0714 - accuracy: 0.9798
Epoch 8/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.0608 - accuracy: 0.9836
Epoch 9/10
1800/1800 [==============================] - 2s 1ms/step - loss: 0.0524 - accuracy: 0.9857
Epoch 10/10
200/200 [==============================] - 0s 869us/step
Epoch 1/10
18

#### 8- Based on your results, what is the best batch_size and epochs?   Now, test your model with the best batch_size and epochs on the testing set.   grid.best_estimator_.model  gives you the best model found and trained in the grid-search. What is the prediction accuracy on the testing set?

In [152]:
# Best params
best_params = grid_result.best_params_

print("The best batch size is {} and the best epochs is {}".format(best_params['batch_size'], best_params['epochs']))

The best batch size is 30 and the best epochs is 15


#### The best batch size is 30 and the best epochs is 15

In [153]:
# Get the best model
best_model = grid_result.best_estimator_.model_

In [154]:
# Prediction:
y_predict = best_model.predict(X_test, verbose=1)
print (y_predict.shape)

313/313 [==============================] - 0s 865us/step
(10000, 10)


In [155]:
score = best_model.evaluate(X_test, y_test, verbose=1)
print('The accuracy is: ', score[1])

313/313 [==============================] - 0s 954us/step - loss: 0.0769 - accuracy: 0.9758
The accuracy is:  0.9757999777793884


#### The accuracy is:  0.9757999777793884